In [ ]:
import sys, json
sys.path.insert(1, '../')
from collection import *

import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
df_gtemp = pd.read_csv('global_temp.csv')
df_gtemp['date'] = pd.to_datetime(df_gtemp['date'])

df_gco2 = pd.read_csv('global_co2.csv')
df_gco2['year'] = pd.to_datetime(df_gco2['year'])

with open('../data_collection/countries.json', 'r', encoding = 'latin1') as f:
    countries_json = json.load(f)
country_alpha3 = [[country, countries_json[country]['alpha3']] for country in countries_json.keys()]
df_country_alpha3 = pd.DataFrame(country_alpha3, columns = ['country', 'alpha3'])
df_gtemp = df_gtemp.merge(df_country_alpha3, on = 'country')

df_gtemp = df_gtemp[df_gtemp.date.dt.year > 1849].reset_index(drop = True)
df_gtemp['yearly_mean'] = df_gtemp.groupby(by = ['country', df_gtemp.date.dt.year])['monthly_anomaly'].transform('mean')
df_gtemp['rolling_5y'] = df_gtemp.groupby(by = 'country')['monthly_anomaly'].rolling(window = 60, min_periods = 48).mean().reset_index().loc[:, 'monthly_anomaly'] # 60 months = 5 years, min rquired 4 years
df_gtemp_yearly = df_gtemp.groupby(by = ['country', df_gtemp.date.dt.year]).first().drop(columns = ['date', 'monthly_anomaly']).reset_index()
df_gtemp

In [ ]:
def plot_global_temp_choro(df_gtemp_yearly):
    yearly_min = df_gtemp_yearly.loc[:, 'yearly_mean'].min()
    yearly_max = df_gtemp_yearly.loc[:, 'yearly_mean'].max()

    fig = px.choropleth(data_frame = df_gtemp_yearly,
                        title = '<br>Global Temperature',
                        locations = 'alpha3',
                        color = 'yearly_mean',  # value in column 'Confirmed' determines color
                        hover_name = 'country',
                        color_continuous_scale = 'Jet',  #  color scale red, yellow green
                        range_color = [yearly_min, yearly_max],
                        animation_frame = 'date',
                        width = 1400, 
                        height = 1000)

    fig.update_layout(
        margin = dict(l = 70, r = 70, t = 20, b = 0, pad = 4),
        paper_bgcolor = 'LightSteelBlue',
        title_font_size = 50, 
        coloraxis_colorbar = dict(
            title = 'Berkeley Earth<br>Temperature',
            thicknessmode = 'pixels', thickness = 40,
            lenmode = 'pixels', len = 620,
            ticks = 'outside', 
            ticksuffix = r' °C',
            dtick = 1),
        title = {
            "yref": "paper",
            "y" : 1,
            "yanchor" : "bottom"})

    fig.show()
    
plot_global_temp_choro(df_gtemp_yearly)

In [ ]:
def plot_selective_lineplot(df, measuring_unit):
    """
    Plots a linechart for either countrywise Berkeley Earth Temperature or CO_2
    
    :param df: pd.DataFrame, data to plot
    :param measuring_unit: str, either 'temp' to plot temperature or 'co2' to plot global co2
    """
    if measuring_unit == 'temp':
        x, y = 'date', 'monthly_anomaly'
        color = '#DC3220'
        title_prefix = 'Berkeley Earth Temperature'
        yaxis_title = 'Temperature °C'
    elif measuring_unit == 'co2':
        x, y = 'year', 'co2'
        color = '#000000'
        title_prefix = 'Yearly CO<sub>2</sub> Emission'
        yaxis_title = 'MtCO<sub>2</sub>'
    else:
        raise ValueError("Only 'temp' or 'co2' is a valid value for `measuring_unit`")

    fig = go.FigureWidget(layout = {'height': 600})
    countries = list(df.loc[:, 'country'].unique())
    buttons = []
    
    first, vis = True, True
    for i in range(len(countries)):
        country = countries[i]
        data = df[df.loc[:, 'country'] == country]
        fig.add_trace(
            go.Scatter(
                x = data.loc[:, x],
                y = data.loc[:, y],
                name = country,
                line = dict(color = color),
                visible = vis,
            ))
        if measuring_unit == 'temp':
            fig.add_trace(
                go.Scatter(
                    x = data.loc[:, x],
                    y = data.loc[:, 'rolling_5y'],
                    name = country + ' smoothed',
                    line = dict(color = color),
                    visible = vis,
                ))
        if first:
            first, vis = False, False
            fig.update_layout(title = title_prefix + ' of ' + country)
        
        if measuring_unit == 'temp':
            visible = [False] * 2 * len(countries)
            visible[2 * i] = True
            visible[2 * i + 1] = True
        else:
            visible = [False] * len(countries)
            visible[i] = True
        new_button = dict(label = country,
                          method = 'update',
                          args = [{'visible': visible},
                                # the index of True aligns with the indices of plot traces
                                {'title': title_prefix + ' of ' + country,
                                 'showlegend': True}])
        buttons.append(new_button)

    # Add dropdown
    fig.update_layout(
        updatemenus = [go.layout.Updatemenu(
            active = 0,
            buttons = buttons,
            direction = 'down',
            pad = {'r': 10, 't': 10},
            showactive = True,
            x = 1,
            xanchor = 'right',
            y = 1,
            yanchor = 'bottom')],
        xaxis_title = 'Time',
        yaxis_title = yaxis_title,
    )

    # Add range slider
    fig.update_layout(
        xaxis = dict(
            rangeselector = dict(
                buttons = list([
                    dict(count = 1,
                         label = '1y',
                         step = 'year',
                         stepmode = 'backward'),
                    dict(step = 'all',
                         label = 'all')
                ])
            ),
            rangeslider = dict(visible = True),
            type = 'date'
        )
    )

    fig.show()


In [ ]:
plot_selective_lineplot(df_gtemp, 'temp')

In [ ]:
plot_selective_lineplot(df_gco2, 'co2')

In [ ]:
def plot_global_co2(df):
    """
    Plots the global CO_2 emmisions in a line chart
    
    :param df: pd.DataFrame, data to plot
    """
    fig = go.FigureWidget(layout = {'height': 600})
    
    fig.add_trace(
        go.Scatter(
            x = df.loc[:, 'year'],
            y = df.loc[:, 'co2'],
            name = 'Global CO<sub>2</sub>',
            line = dict(color = '#000000')
        ))

    # Add range slider
    fig.update_layout(
        xaxis_title = 'Time',
        yaxis_title = 'MtCO<sub>2</sub>',
        title = 'Yearly Global CO<sub>2</sub> Emission',
        xaxis = dict(
            rangeselector = dict(
                buttons = list([
                    dict(count = 1,
                         label = '1y',
                         step = 'year',
                         stepmode = 'backward'),
                    dict(step = 'all',
                         label = 'all')
                ])
            ),
            rangeslider = dict(visible = True),
            type = 'date'
        )
    )

    fig.show()


# fill the missing 'co2' values of each country with countrywise co2.mean() to minimize global error
df_gco2_filled = df_gco2.copy()
df_gco2_filled['co2'] = df_gco2_filled.groupby('country').transform(lambda x: x.fillna(x.mean()))['co2']
df_global_co2 = df_gco2_filled.groupby(by = 'year').sum().reset_index()
plot_global_co2(df_global_co2)